**Naufal Rasyid S - Internship Project**

# ETANA

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

Melakukan cek apakah website bisa di akses

In [ ]:
# URLwebsite
base_url = "https://www.id.etanabiotech.com/siaran-pers"  # Ganti dengan URL website Anda

# URL robots.txt
robots_url = f"{base_url}/robots.txt"

# Periksa isi robots.txt
response = requests.get(robots_url)
if response.status_code == 200:
    print("Isi robots.txt:")
    print(response.text)
else:
    print(f"Tidak dapat mengakses robots.txt, status code: {response.status_code}")

Tidak dapat mengakses robots.txt, status code: 404


FILTER BASED ON YEAR 2024-2023

In [ ]:
# Base URL for the website
BASE_URL = "https://www.id.etanabiotech.com"

# Headers to mimic browser behavior
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.110 Safari/537.36"
}

# Function to scrape article details
def scrape_news_detail(article_url):
    try:
        response = requests.get(article_url, headers=HEADERS)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching article URL {article_url}: {e}")
        return "Failed to fetch article content"

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the article content; adjust selector as necessary
    content_section = soup.select_one('.article-content')  # Adjust to match the website's structure
    if content_section:
        return content_section.get_text(strip=True)

    # Fallback for extracting text from paragraphs
    paragraphs = soup.find_all('p')
    if paragraphs:
        return ' '.join(p.get_text(strip=True) for p in paragraphs)

    return "No content found"

# Function to scrape the list of articles from a single page
def scrape_news_list(page_url, allowed_years):
    try:
        response = requests.get(page_url, headers=HEADERS)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching page URL {page_url}: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all article entries
    articles = soup.select('.BlogList-item')  # Adjust this selector based on the website structure
    if not articles:
        print(f"No articles found on page: {page_url}")
        return []

    news_list = []
    print(f"Found {len(articles)} articles on {page_url}.")

    for article in articles:
        # Extract title
        title = article.select_one('.BlogList-item-title')
        title_text = title.get_text(strip=True) if title else "No Title"

        # # Extract date format = 10-12-2024
        # date_tag = article.select_one('time')
        # date = date_tag['datetime'] if date_tag and 'datetime' in date_tag.attrs else "No Date"

        # Extract date
        date_tag = article.select_one('time')
        date = (datetime.strptime(date_tag['datetime'], "%Y-%m-%d").strftime("%d %B %Y")
            if date_tag and 'datetime' in date_tag.attrs else "No Date")

        # Filter by year
        if any(str(year) in date for year in allowed_years):
            # Extract link
            link_tag = article.select_one('a')
            relative_link = link_tag['href'].strip() if link_tag else None
            full_link = BASE_URL + relative_link if relative_link and relative_link.startswith('/') else relative_link

            # Fetch detailed content
            content = scrape_news_detail(full_link) if full_link else "No Content Available"

            # Append to the news list
            news_list.append({
                'Title': title_text,
                'Content': content,
                'Source': "Etana",
                'Date': date,
            })

    return news_list

# Main function to scrape multiple pages using `offset`
def scrape_all_pages(start_url, offsets, allowed_years):
    all_articles = []
    for offset in offsets:
        # Construct page URL dynamically
        page_url = f"{start_url}?offset={offset}" if offset else start_url
        print(f"Scraping page: {page_url}")

        # Scrape articles from the page
        articles = scrape_news_list(page_url, allowed_years)
        if not articles:
            print(f"No articles found on page with offset {offset}.")
            continue

        # Append results from the page
        all_articles.extend(articles)

    return all_articles
    # return pd.DataFrame(all_articles)

In [ ]:
# Main execution
if __name__ == "__main__":
    # URL of the press release page
    press_url = f"{BASE_URL}/siaran-pers"

    # Define offsets for pagination
    offsets = [None, "1710399600052", "1693735200635"]  # Add additional offsets for more pages

    # Allowed years for filtering
    allowed_years = [2023, 2024]

    # Scrape articles from all specified offsets
    articles = scrape_all_pages(press_url, offsets, allowed_years)

    if articles:
        # Save to a single DataFrame
        df = pd.DataFrame(articles)
        df.to_csv('Etana.csv', index=False)
        print("Scraped data from all pages saved to 'Etana.csv'")
        print(df)  # Display combined DataFrame
    else:
        print("No articles to save.")

Scraping page: https://www.id.etanabiotech.com/siaran-pers
Found 6 articles on https://www.id.etanabiotech.com/siaran-pers.
Scraping page: https://www.id.etanabiotech.com/siaran-pers?offset=1710399600052
Found 6 articles on https://www.id.etanabiotech.com/siaran-pers?offset=1710399600052.
Scraping page: https://www.id.etanabiotech.com/siaran-pers?offset=1693735200635
Found 6 articles on https://www.id.etanabiotech.com/siaran-pers?offset=1693735200635.
Scraped data from all pages saved to 'Etana.csv'
                                                Title  \
0   ETANA LUNCURKAN TERAPI KANKER INOVATIF DI INDO...   
1   ETANA DAN CELLTRION TANDATANGANI KESEPAKATAN S...   
2   KUNJUNGI ETANA, BPOM DUKUNG INDUSTRI BIOTEKNOL...   
3   ETANA BERSAMA DENGAN RSPAD GATOT SUBROTO SELEN...   
4   ETANA DUKUNG RS PENDIDIKAN UNIVERSITAS SYIAH K...   
5   ETANA DAN PRIMAKU JALIN KERJASAMA KOLABORASI S...   
6   ETANA JALIN KERJA SAMA STRATEGIS DENGAN BEIGEN...   
7   FASILITAS PRODUKSI VAKSIN mRNA ETAN

# ETANA ENGLISH Ver

In [ ]:
# Base URL for the website
BASE_URL = "https://www.id.etanabiotech.com"

# Headers to mimic browser behavior
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.110 Safari/537.36"
}

# Function to scrape article details
def scrape_news_detail(article_url):
    try:
        response = requests.get(article_url, headers=HEADERS)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching article URL {article_url}: {e}")
        return "Failed to fetch article content"

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the article content; adjust selector as necessary
    content_section = soup.select_one('.article-content')  # Adjust to match the website's structure
    if content_section:
        return content_section.get_text(strip=True)

    # Fallback for extracting text from paragraphs
    paragraphs = soup.find_all('p')
    if paragraphs:
        return ' '.join(p.get_text(strip=True) for p in paragraphs)

    return "No content found"

# Function to scrape the list of articles from a single page
def scrape_news_list(page_url, allowed_years):
    try:
        response = requests.get(page_url, headers=HEADERS)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching page URL {page_url}: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all article entries
    articles = soup.select('.BlogList-item')  # Adjust this selector based on the website structure
    if not articles:
        print(f"No articles found on page: {page_url}")
        return []

    news_list = []
    print(f"Found {len(articles)} articles on {page_url}.")

    for article in articles:
        # Extract title
        title = article.select_one('.BlogList-item-title')
        title_text = title.get_text(strip=True) if title else "No Title"

        # # Extract date format = 10-12-2024
        # date_tag = article.select_one('time')
        # date = date_tag['datetime'] if date_tag and 'datetime' in date_tag.attrs else "No Date"

        # Extract date
        date_tag = article.select_one('time')
        date = (datetime.strptime(date_tag['datetime'], "%Y-%m-%d").strftime("%d %B %Y")
            if date_tag and 'datetime' in date_tag.attrs else "No Date")

        # Filter by year
        if any(str(year) in date for year in allowed_years):
            # Extract link
            link_tag = article.select_one('a')
            relative_link = link_tag['href'].strip() if link_tag else None
            full_link = BASE_URL + relative_link if relative_link and relative_link.startswith('/') else relative_link

            # Fetch detailed content
            content = scrape_news_detail(full_link) if full_link else "No Content Available"

            # Append to the news list
            news_list.append({
                'Title': title_text,
                'Content': content,
                'Source': "Etana",
                'Date': date,
            })

    return news_list

# Main function to scrape multiple pages using `offset`
def scrape_all_pages(start_url, offsets, allowed_years):
    all_articles = []
    for offset in offsets:
        # Construct page URL dynamically
        page_url = f"{start_url}?offset={offset}" if offset else start_url
        print(f"Scraping page: {page_url}")

        # Scrape articles from the page
        articles = scrape_news_list(page_url, allowed_years)
        if not articles:
            print(f"No articles found on page with offset {offset}.")
            continue

        # Append results from the page
        all_articles.extend(articles)

    return all_articles

In [ ]:
# Main execution
if __name__ == "__main__":
    # URL of the press release page
    press_url = f"{BASE_URL}/news"

    # Define offsets for pagination
    offsets = [None, "1710399600052", "1693735200635"]  # Add additional offsets for more pages

    # Allowed years for filtering
    allowed_years = [2023, 2024]

    # Scrape articles from all specified offsets
    articles = scrape_all_pages(press_url, offsets, allowed_years)

    if articles:
        # Save to a single DataFrame
        df = pd.DataFrame(articles)
        df.to_csv('Etana.csv', index=False)
        print("Scraped data from all pages saved to 'Etana.csv'")
        print(df)  # Display combined DataFrame
    else:
        print("No articles to save.")

Scraping page: https://www.id.etanabiotech.com/news
Found 6 articles on https://www.id.etanabiotech.com/news.
Scraping page: https://www.id.etanabiotech.com/news?offset=1710399600052
Found 6 articles on https://www.id.etanabiotech.com/news?offset=1710399600052.
Scraping page: https://www.id.etanabiotech.com/news?offset=1693735200635
Found 6 articles on https://www.id.etanabiotech.com/news?offset=1693735200635.
Scraped data from all pages saved to 'Etana.csv'
                                                Title  \
0   ETANA LAUNCHES INNOVATIVE CANCER THERAPIES IN ...   
1   ETANA AND CELLTRION SIGN STRATEGIC AGREEMENT F...   
2   ETANA ESTABLISHES STRATEGIC COOPERATION WITH B...   
3   ETANA'S MRNA VACCINE PRODUCTION FACILITY RECEI...   
4   ETANA STRENGTHENS BIOPHARMACEUTICAL COOPERATIO...   
5   ETANA STRENGTHENS COOPERATION WITH BRIN AND UN...   
6   ETANA ESTABLISHES STRATEGIC COOPERATION WITH B...   
7   ETANA'S MRNA VACCINE PRODUCTION FACILITY RECEI...   
8   ETANA STRENGTHENS BI

# SOHO

In [ ]:
# URLwebsite
base_url = "https://www.sohoglobalhealth.com"  # Ganti dengan URL website Anda

# URL robots.txt
robots_url = f"{base_url}/robots.txt"

# Periksa isi robots.txt
response = requests.get(robots_url)
if response.status_code == 200:
    print("Isi robots.txt:")
    print(response.text)
else:
    print(f"Tidak dapat mengakses robots.txt, status code: {response.status_code}")

Isi robots.txt:
# See http://www.robotstxt.org/robotstxt.html for documentation on how to use the robots.txt file



In [ ]:
url = 'https://www.sohoglobalhealth.com/en/newsrooms'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [ ]:
print(soup)

<!DOCTYPE html>
<html>
<head>
<title>SOHO Global Health</title>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<meta content="authenticity_token" name="csrf-param"/>
<meta content="AMnVvDPh3B+0Z1MRiKFIDDce7TaxDeZQEH6NlfsyOCz4YOY/s9DUhpu3YAlOk3avGG2kZ+rjzdFXAHoVvVGkWg==" name="csrf-token"/>
<link crossorigin="anonymous" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css" integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/css?family=Didact+Gothic|Open+Sans:300,400,700" rel="stylesheet" type="text/css"/>
<link href="https://fonts.googleapis.com/css?family=Catamaran:400,500,600,700" rel="stylesheet" type="text/css"/>
<link crossorigin="anonymous" data-turbolinks-track="reload" href="/assets/application-00a988e7bed2160c2f1b62af97498cfa88e0c6b0faea1c3faf23f8a72c0f5aef.css" integrity="sha256-AKmI577SFgwvG2Kvl0m

FILTER BASED ON YEAR 2024 - 2023

In [ ]:
# Base URL for the website
BASE_URL = "https://www.sohoglobalhealth.com"

# Headers to mimic browser behavior
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.110 Safari/537.36"
}

# Function to scrape the details of an individual article
def scrape_article_detail(relative_url):
    # Ensure the URL is properly formed
    if relative_url.startswith("newsrooms"):
        article_url = f"{BASE_URL}/id/{relative_url.strip()}"  # Adjusted to include '/id/'
    elif relative_url.startswith("/"):
        article_url = f"{BASE_URL}{relative_url.strip()}"
    elif relative_url.startswith("http"):
        article_url = relative_url.strip()
    else:
        print(f"Invalid URL format: {relative_url}")
        return "Invalid URL format"

    try:
        response = requests.get(article_url, headers=HEADERS, allow_redirects=True)
        response.raise_for_status()  # Check for HTTP errors
    except requests.TooManyRedirects:
        print(f"Too many redirects for URL: {article_url}")
        return "Failed due to too many redirects"
    except requests.RequestException as e:
        print(f"Error fetching article: {article_url}, Error: {e}")
        return "Failed to fetch article content"

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract content; adjust selectors based on article page structure
    paragraphs = soup.find_all('p')
    content = ' '.join(p.get_text(strip=True) for p in paragraphs) if paragraphs else "No content found"
    return content

# Function to scrape the list of articles from the newsroom page
def scrape_news_list(page_url):
    try:
        response = requests.get(page_url, headers=HEADERS)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching page URL: {page_url}, Error: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the list of articles
    articles = soup.select('.news-list.current > ul > li')
    if not articles:
        print("No articles found.")
        return []

    news_list = []
    allowed_years = ['2023', '2024']  # Allowed years for filtering

    for article in articles:
        # Extract relative link
        link_tag = article.find('a')
        relative_link = link_tag['href'].strip() if link_tag and 'href' in link_tag.attrs else None

        # Extract date
        date_tag = article.select_one('.news-list-title-wrapper > p')
        date = date_tag.get_text(strip=True) if date_tag else "No Date"

        # Filter by allowed years
        if not any(year in date for year in allowed_years):
            continue  # Skip articles that don't match the allowed years

        # Extract title
        title_tag = article.select_one('.news-list-title-wrapper > h1')
        title = title_tag.get_text(strip=True) if title_tag else "No Title"

        # Fetch detailed content
        if relative_link:
            content = scrape_article_detail(relative_link)
        else:
            content = "No Content Available"

        # Add to list
        news_list.append({
            'Title': title,
            'Content': content,
            'Date': date,
            'Source': "Soho Global Health"
        })

    return news_list

In [ ]:
# Main function to scrape the newsroom section
if __name__ == "__main__":
    newsroom_url = f"{BASE_URL}/id/newsrooms"

    # Scrape articles from the newsroom page
    articles = scrape_news_list(newsroom_url)

    if articles:
        # Save to a DataFrame
        df = pd.DataFrame(articles)
        df.to_csv('Soho.csv', index=False)
        print("Scraped data saved to 'Soho.csv'")
        print(df)  # Display first few rows
    else:
        print("No articles to save.")

Scraped data saved to 'Soho.csv'
                                                Title  \
0   PT Parit Padang Global (PPG) Tandatangani Perj...   
1   Ceremonial Kick-Off Proyek Solusi Manajemen Tr...   
2   PT Parit Padang Global (PPG) Tandatangani Perj...   
3   Penandatanganan Kerjasama PPGOS Subdistributor...   
4   Seremonial Penandatanganan MoU Pendanaan RISPR...   
5   HKN 2023: SOHO Gelar Talkshow Inovasi Temulawa...   
6   SOHO Raih Penghargaan Karya Anak Bangsa 2023 d...   
7   PT Parit Padang Global (PPG) Tandatangani Perj...   
8   SOHO Global Health Tbk dianugerahi Medali Peru...   
9   PT Soho Industri Pharmasi meraih Peringkat ke-...   
10  SOHO dan IPB Mendapatkan Sertifikat Patent Pro...   
11  FITKOM bersama Guardian Dukung Anak Tumbuh Seh...   
12  PT Parit Padang Global (PPG) Tandatangani Perj...   
13  PT Parit Padang Global (PPG) Tandatangani Perj...   
14  PT Parit Padang Global (PPG) Menandatangani Pe...   
15  PT Soho Global Health Tbk menerima PPKM Award ...  

# SOHO ENGLISH Ver

In [ ]:
# Base URL for the website
BASE_URL = "https://www.sohoglobalhealth.com"

# Headers to mimic browser behavior
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.110 Safari/537.36"
}

# Function to scrape the details of an individual article
def scrape_article_detail(relative_url):
    # Ensure the URL is properly formed
    if relative_url.startswith("newsrooms"):
        article_url = f"{BASE_URL}/en/{relative_url.strip()}"  # Adjusted to include '/id/'
    elif relative_url.startswith("/"):
        article_url = f"{BASE_URL}{relative_url.strip()}"
    elif relative_url.startswith("http"):
        article_url = relative_url.strip()
    else:
        print(f"Invalid URL format: {relative_url}")
        return "Invalid URL format"

    try:
        response = requests.get(article_url, headers=HEADERS, allow_redirects=True)
        response.raise_for_status()  # Check for HTTP errors
    except requests.TooManyRedirects:
        print(f"Too many redirects for URL: {article_url}")
        return "Failed due to too many redirects"
    except requests.RequestException as e:
        print(f"Error fetching article: {article_url}, Error: {e}")
        return "Failed to fetch article content"

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract content; adjust selectors based on article page structure
    paragraphs = soup.find_all('p')
    content = ' '.join(p.get_text(strip=True) for p in paragraphs) if paragraphs else "No content found"
    return content

# Function to scrape the list of articles from the newsroom page
def scrape_news_list(page_url):
    try:
        response = requests.get(page_url, headers=HEADERS)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching page URL: {page_url}, Error: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the list of articles
    articles = soup.select('.news-list.current > ul > li')
    if not articles:
        print("No articles found.")
        return []

    news_list = []
    allowed_years = ['2023', '2024']  # Allowed years for filtering

    for article in articles:
        # Extract relative link
        link_tag = article.find('a')
        relative_link = link_tag['href'].strip() if link_tag and 'href' in link_tag.attrs else None

        # Extract date
        date_tag = article.select_one('.news-list-title-wrapper > p')
        date = date_tag.get_text(strip=True) if date_tag else "No Date"

        # Filter by allowed years
        if not any(year in date for year in allowed_years):
            continue  # Skip articles that don't match the allowed years

        # Extract title
        title_tag = article.select_one('.news-list-title-wrapper > h1')
        title = title_tag.get_text(strip=True) if title_tag else "No Title"

        # Fetch detailed content
        if relative_link:
            content = scrape_article_detail(relative_link)
        else:
            content = "No Content Available"

        # Add to list
        news_list.append({
            'Title': title,
            'Content': content,
            'Date': date,
            'Source': "Soho Global Health"
        })

    return news_list

In [ ]:
# Main function to scrape the newsroom section
if __name__ == "__main__":
    newsroom_url = f"{BASE_URL}/en/newsrooms"

    # Scrape articles from the newsroom page
    articles = scrape_news_list(newsroom_url)

    if articles:
        # Save to a DataFrame
        df = pd.DataFrame(articles)
        df.to_csv('Soho.csv', index=False)
        print("Scraped data saved to 'Soho.csv'")
        print(df)  # Display first few rows
    else:
        print("No articles to save.")

Scraped data saved to 'Soho.csv'
                                                Title  \
0   PT Parit Padang Global (PPG) Signs Distributio...   
1   Ceremonial Project Kick-Off Transport and Disp...   
2   PT Parit Padang Global (PPG) Signs Distributio...   
3   Signing of the Cooperation Agreement for PPGOS...   
4   Ceremonial signing of the MoU for the Funding ...   
5   HKN 2023: SOHO Organizes a Talk Show on Temula...   
6   SOHO has been awarded the “Penghargaan Karya A...   
7   PT Parit Padang Global (PPG) Signs Distributio...   
8   SOHO Global Health Tbk was awarded a Bronze Me...   
9   PT Soho Industri Pharmasi achieved 3rd place (...   
10  SOHO and IPB Obtain Patent Certificate for Fun...   
11  Fitkom and Guardian support children to grow h...   
12  PT Parit Padang Global (PPG) has signed a Dist...   
13  PT Parit Padang Global (PPG) has signed a Dist...   
14  PT Parit Padang Global (PPG) has signed a Dist...   
15  PT Soho Global Health Tbk Received PPKM Award ...  

In [ ]:
df

,Title,Content,Date,Source
0,PT Parit Padang Global (PPG) Signs Distributio...,12 August 2024 12 June 2024 06 June 2024 12 Au...,12 August 2024,Soho Global Health
1,Ceremonial Project Kick-Off Transport and Disp...,12 August 2024 12 June 2024 06 June 2024 12 Ju...,12 June 2024,Soho Global Health
2,PT Parit Padang Global (PPG) Signs Distributio...,12 August 2024 12 June 2024 06 June 2024 06 Ju...,06 June 2024,Soho Global Health
3,Signing of the Cooperation Agreement for PPGOS...,12 August 2024 12 June 2024 06 June 2024 24 Ma...,24 May 2024,Soho Global Health
4,Ceremonial signing of the MoU for the Funding ...,12 August 2024 12 June 2024 06 June 2024 02 Fe...,02 February 2024,Soho Global Health
5,HKN 2023: SOHO Organizes a Talk Show on Temula...,12 August 2024 12 June 2024 06 June 2024 14 No...,14 November 2023,Soho Global Health
6,SOHO has been awarded the “Penghargaan Karya A...,12 August 2024 12 June 2024 06 June 2024 13 No...,13 November 2023,Soho Global Health
7,PT Parit Padang Global (PPG) Signs Distributio...,12 August 2024 12 June 2024 06 June 2024 27 Oc...,27 October 2023,Soho Global Health
8,SOHO Global Health Tbk was awarded a Bronze Me...,12 August 2024 12 June 2024 06 June 2024 20 Oc...,20 October 2023,Soho Global Health
9,PT Soho Industri Pharmasi achieved 3rd place (...,12 August 2024 12 June 2024 06 June 2024 21 Ju...,21 July 2023,Soho Global Health


# Traknus

In [ ]:
# URLwebsite
base_url = "https://www.traknus.co.id"  # Ganti dengan URL website Anda

# URL robots.txt
robots_url = f"{base_url}/robots.txt"

# Periksa isi robots.txt
response = requests.get(robots_url)
if response.status_code == 200:
    print("Isi robots.txt:")
    print(response.text)
else:
    print(f"Tidak dapat mengakses robots.txt, status code: {response.status_code}")

Isi robots.txt:
User-agent: *
Disallow:

Sitemap: https://www.traknus.co.id/sitemap.xml


In [ ]:
url = 'https://www.traknus.co.id/id/news/category/news/'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [ ]:
print(soup)

<!DOCTYPE html>
<html lang="en">
<head>
<!-- Bootstrap core CSS -->
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0,shrink-to-fit=no" name="viewport"/>
<!-- Locale -->
<meta content="id" http-equiv="Content-Language"/>
<!-- To the Future! -->
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<!-- Meta Tags -->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="PyroCMS" name="generator"/>
<meta content="Traktor, Harga Traktor, Traktor Nusantara, Traktor Tangan, Alat Berat Tambang, Jenis Alat Berat, Sewa Traktor Pertanian, Traktor Indonesia, Traktor Terbaru, Jual Traktor Murah" name="keywords"/>
<meta content="9rnjgj9dfl7pgx9olbz7pmf4dn9an6" name="facebook-domain-verification"/>
<!-- Favicon -->
<meta content="ESrSFipPQdd8bt1n4-Gf_gToyvTMS1RA5cd1lO457Ag" name="google-site-verification"/> <title>Traktor Nusantara | News</title>
<meta content="" name="description"/>
<!-- Favicon -->
<link href="/

In [ ]:
# # Base URL for the Traknus website
# BASE_URL = "https://www.traknus.co.id/id/news/category/news"

# # Headers to mimic browser behavior
# HEADERS = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.110 Safari/537.36"
# }

# # Function to scrape article details
# def scrape_news_detail(article_url):
#     try:
#         response = requests.get(article_url, headers=HEADERS)
#         response.raise_for_status()
#     except requests.RequestException as e:
#         print(f"Error fetching article URL {article_url}: {e}")
#         return "Failed to fetch article content"

#     soup = BeautifulSoup(response.text, 'html.parser')

#     # Extract the article content
#     content_section = soup.find("div", class_="newsContent wysiwygContent")
#     if content_section:
#         return content_section.get_text(separator="\n", strip=True)

#     return "No content found"

# # Function to scrape the list of articles from a single page
# def scrape_news_list(page_url):
#     try:
#         response = requests.get(page_url, headers=HEADERS)
#         response.raise_for_status()
#     except requests.RequestException as e:
#         print(f"Error fetching page URL {page_url}: {e}")
#         return []

#     soup = BeautifulSoup(response.text, 'html.parser')

#     # Find all regular article entries
#     small_articles = soup.find_all("div", class_="newsBoxSmall")
#     big_article = soup.find("div", class_="newsBoxBig")  # Highlighted article

#     if not small_articles and not big_article:
#         print(f"No articles found on page: {page_url}")
#         return []

#     news_list = []

#     # Process the highlighted article (newsBoxBig)
#     if big_article:
#         try:
#             title = big_article.find("h2", class_="title").text.strip()
#             date = big_article.find("span", class_="date").text.strip()
#             link_tag = big_article.find("a", href=True)
#             full_link = link_tag['href'].strip() if link_tag else None
#             content = scrape_news_detail(full_link) if full_link else "No Content Available"

#             news_list.append({
#                 'Title': title,
#                 'Content': content,
#                 'Source': "Traktor Nusantara",
#                 'Date': date,
#             })
#         except Exception as e:
#             print(f"Error processing big article: {e}")

#     # Process regular articles (newsBoxSmall)
#     for article in small_articles:
#         try:
#             title = article.find("h2", class_="title").text.strip()
#             date = article.find("span", class_="date").text.strip()
#             link_tag = article.find("a", href=True)
#             relative_link = link_tag['href'].strip() if link_tag else None
#             full_link = BASE_URL + relative_link if relative_link and relative_link.startswith('/') else relative_link
#             content = scrape_news_detail(full_link) if full_link else "No Content Available"

#             news_list.append({
#                 'Title': title,
#                 'Content': content,
#                 'Source': "Traktor Nusantara",
#                 'Date': date,
#             })
#         except Exception as e:
#             print(f"Error processing small article: {e}")

#     return news_list

# # Function to scrape multiple pages using `page`
# def scrape_all_pages(start_url, num_pages):
#     all_articles = []
#     for page in range(1, num_pages + 1):
#         if page == 1:
#             page_url = start_url
#         else:
#             page_url = f"{start_url}?page={page}"
#         print(f"Scraping page: {page_url}")

#         # Scrape articles from the page
#         articles = scrape_news_list(page_url)
#         if not articles:
#             print(f"No articles found on page {page}.")
#             continue

#         # Append results from the page
#         all_articles.extend(articles)

#     return all_articles

In [ ]:
# # Main execution
# if __name__ == "__main__":
#     # Scrape articles from all specified pages
#     articles = scrape_all_pages(BASE_URL, 10)  # Adjust num_pages as needed

#     if articles:
#         # Save to a single DataFrame
#         df = pd.DataFrame(articles)
#         df.to_csv('traknus_news_all_pages.csv', index=False)
#         print("Scraped data from all pages saved to 'traknus_news_all_pages.csv'")
#         print(df)  # Display combined DataFrame
#     else:
#         print("No articles to save.")

FILTER BY YEAR 2024 - 2023

In [ ]:
# Base URL for the Traknus website
BASE_URL = "https://www.traknus.co.id/id/news/category/news"

# Headers to mimic browser behavior
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.110 Safari/537.36"
}

# Function to scrape article details
def scrape_news_detail(article_url):
    try:
        response = requests.get(article_url, headers=HEADERS)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching article URL {article_url}: {e}")
        return "Failed to fetch article content"

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the article content
    content_section = soup.find("div", class_="newsContent wysiwygContent")
    if content_section:
        return content_section.get_text(separator="\n", strip=True)

    return "No content found"

# Function to scrape the list of articles from a single page
def scrape_news_list(page_url):
    try:
        response = requests.get(page_url, headers=HEADERS)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching page URL {page_url}: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all regular article entries
    small_articles = soup.find_all("div", class_="newsBoxSmall")
    big_article = soup.find("div", class_="newsBoxBig")  # Highlighted article

    if not small_articles and not big_article:
        print(f"No articles found on page: {page_url}")
        return []

    news_list = []

    # Process the highlighted article (newsBoxBig)
    if big_article:
        try:
            title = big_article.find("h2", class_="title").text.strip()
            date = big_article.find("span", class_="date").text.strip()
            link_tag = big_article.find("a", href=True)
            full_link = link_tag['href'].strip() if link_tag else None
            content = scrape_news_detail(full_link) if full_link else "No Content Available"

            # Filter articles by year
            if "2023" in date or "2024" in date:
                news_list.append({
                    'Title': title,
                    'Content': content,
                    'Source': "Traktor Nusantara",
                    'Date': date,
                })
        except Exception as e:
            print(f"Error processing big article: {e}")

    # Process regular articles (newsBoxSmall)
    for article in small_articles:
        try:
            title = article.find("h2", class_="title").text.strip()
            date = article.find("span", class_="date").text.strip()
            link_tag = article.find("a", href=True)
            relative_link = link_tag['href'].strip() if link_tag else None
            full_link = BASE_URL + relative_link if relative_link and relative_link.startswith('/') else relative_link
            content = scrape_news_detail(full_link) if full_link else "No Content Available"

            # Filter articles by year
            if "2023" in date or "2024" in date:
                news_list.append({
                    'Title': title,
                    'Content': content,
                    'Source': "Traktor Nusantara",
                    'Date': date,
                })
        except Exception as e:
            print(f"Error processing small article: {e}")

    return news_list

# Function to scrape multiple pages using `page`
def scrape_all_pages(start_url, num_pages):
    all_articles = []
    for page in range(1, num_pages + 1):
        if page == 1:
            page_url = start_url
        else:
            page_url = f"{start_url}?page={page}"
        print(f"Scraping page: {page_url}")

        # Scrape articles from the page
        articles = scrape_news_list(page_url)
        if not articles:
            print(f"No articles found on page {page}.")
            continue

        # Append results from the page
        all_articles.extend(articles)

    return all_articles

In [ ]:
# Main execution
if __name__ == "__main__":
    # Scrape articles from all specified pages
    articles = scrape_all_pages(BASE_URL, 10)  # Adjust num_pages as needed

    if articles:
        # Save to a single DataFrame
        df = pd.DataFrame(articles)
        df.to_csv('Traknus.csv', index=False)
        print("Scraped data from all pages saved to 'Traknus.csv'")
        print(df)  # Display combined DataFrame
    else:
        print("No articles to save.")

Scraping page: https://www.traknus.co.id/id/news/category/news
Scraping page: https://www.traknus.co.id/id/news/category/news?page=2
Scraping page: https://www.traknus.co.id/id/news/category/news?page=3
Scraping page: https://www.traknus.co.id/id/news/category/news?page=4
Scraping page: https://www.traknus.co.id/id/news/category/news?page=5
Scraping page: https://www.traknus.co.id/id/news/category/news?page=6
Scraping page: https://www.traknus.co.id/id/news/category/news?page=7
Scraping page: https://www.traknus.co.id/id/news/category/news?page=8
Scraping page: https://www.traknus.co.id/id/news/category/news?page=9
Scraping page: https://www.traknus.co.id/id/news/category/news?page=10
Scraped data from all pages saved to 'Traknus.csv'
                                          Title  \
0                    Genba CHCD 2024 di TRAKNUS   
1   TRAKNUS Group National Management Meetin...   
2   TRAKNUS Group Raih 3 Bronze Winner Dalam...   
3   TRAKNUS Tunjukkan Kepedulian Melalui Don...   


In [ ]:
df

,Title,Content,Source,Date
0,Genba CHCD 2024 di TRAKNUS,Traktor Nusantara (TRAKNUS) menyambut kedatang...,Traktor Nusantara,10 December 2024
1,TRAKNUS Group National Management Meetin...,Mengangkat tema “\nThe Best Industrial & Agric...,Traktor Nusantara,09 December 2024
2,TRAKNUS Group Raih 3 Bronze Winner Dalam...,Traktor Nusantara (TRAKNUS) Group kembali menc...,Traktor Nusantara,06 December 2024
3,TRAKNUS Tunjukkan Kepedulian Melalui Don...,Traktor Nusantara (TRAKNUS) Group kembali meng...,Traktor Nusantara,02 December 2024
4,TRAKNUS Padang Perkuat Kesiapsiagaan Tan...,Traktor Nusantara (TRAKNUS) dalam menanamkan b...,Traktor Nusantara,28 November 2024
...,...,...,...,...
77,PT Traktor Nusantara Luncurkan Green Mat...,"Jakarta (26/05)\n, PT Traktor Nusantara kembal...",Traktor Nusantara,02 June 2023
78,Santunan Yatim dan Dhuafa PT Traktor Nus...,"Jakarta ̶ Di Bulan Suci Ramadhan tahun ini, ...",Traktor Nusantara,19 April 2023
79,PT Traktor Nusantara Raih Tiga Pengharga...,Yogyakarta - PT Traktor Nusantara meraih Tiga ...,Traktor Nusantara,20 March 2023
80,Traktor Nusantara Hadir pada BCA Expover...,"JAKARTA - Pada tahun ini, Traktor Nusantara be...",Traktor Nusantara,01 March 2023


# TRAKNUS ENGLISH Ver

In [ ]:
# Base URL for the Traknus website
BASE_URL = "https://www.traknus.co.id/news/category/news"

# Headers to mimic browser behavior
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.110 Safari/537.36"
}

# Function to scrape article details
def scrape_news_detail(article_url):
    try:
        response = requests.get(article_url, headers=HEADERS)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching article URL {article_url}: {e}")
        return "Failed to fetch article content"

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the article content
    content_section = soup.find("div", class_="newsContent wysiwygContent")
    if content_section:
        return content_section.get_text(separator="\n", strip=True)

    return "No content found"

# Function to scrape the list of articles from a single page
def scrape_news_list(page_url):
    try:
        response = requests.get(page_url, headers=HEADERS)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching page URL {page_url}: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all regular article entries
    small_articles = soup.find_all("div", class_="newsBoxSmall")
    big_article = soup.find("div", class_="newsBoxBig")  # Highlighted article

    if not small_articles and not big_article:
        print(f"No articles found on page: {page_url}")
        return []

    news_list = []

    # Process the highlighted article (newsBoxBig)
    if big_article:
        try:
            title = big_article.find("h2", class_="title").text.strip()
            date = big_article.find("span", class_="date").text.strip()
            link_tag = big_article.find("a", href=True)
            full_link = link_tag['href'].strip() if link_tag else None
            content = scrape_news_detail(full_link) if full_link else "No Content Available"

            # Filter articles by year
            if "2023" in date or "2024" in date:
                news_list.append({
                    'Title': title,
                    'Content': content,
                    'Source': "Traktor Nusantara",
                    'Date': date,
                })
        except Exception as e:
            print(f"Error processing big article: {e}")

    # Process regular articles (newsBoxSmall)
    for article in small_articles:
        try:
            title = article.find("h2", class_="title").text.strip()
            date = article.find("span", class_="date").text.strip()
            link_tag = article.find("a", href=True)
            relative_link = link_tag['href'].strip() if link_tag else None
            full_link = BASE_URL + relative_link if relative_link and relative_link.startswith('/') else relative_link
            content = scrape_news_detail(full_link) if full_link else "No Content Available"

            # Filter articles by year
            if "2023" in date or "2024" in date:
                news_list.append({
                    'Title': title,
                    'Content': content,
                    'Source': "Traktor Nusantara",
                    'Date': date,
                })
        except Exception as e:
            print(f"Error processing small article: {e}")

    return news_list

# Function to scrape multiple pages using `page`
def scrape_all_pages(start_url, num_pages):
    all_articles = []
    for page in range(1, num_pages + 1):
        if page == 1:
            page_url = start_url
        else:
            page_url = f"{start_url}?page={page}"
        print(f"Scraping page: {page_url}")

        # Scrape articles from the page
        articles = scrape_news_list(page_url)
        if not articles:
            print(f"No articles found on page {page}.")
            continue

        # Append results from the page
        all_articles.extend(articles)

    return all_articles

In [ ]:
# Main execution
if __name__ == "__main__":
    # Scrape articles from all specified pages
    articles = scrape_all_pages(BASE_URL, 10)  # Adjust num_pages as needed

    if articles:
        # Save to a single DataFrame
        df = pd.DataFrame(articles)
        df.to_csv('Traknus.csv', index=False)
        print("Scraped data from all pages saved to 'Traknus.csv'")
        print(df)  # Display combined DataFrame
    else:
        print("No articles to save.")

Scraping page: https://www.traknus.co.id/news/category/news
Scraping page: https://www.traknus.co.id/news/category/news?page=2
Scraping page: https://www.traknus.co.id/news/category/news?page=3
Scraping page: https://www.traknus.co.id/news/category/news?page=4
Scraping page: https://www.traknus.co.id/news/category/news?page=5
Scraping page: https://www.traknus.co.id/news/category/news?page=6
Scraping page: https://www.traknus.co.id/news/category/news?page=7
Scraping page: https://www.traknus.co.id/news/category/news?page=8
Scraping page: https://www.traknus.co.id/news/category/news?page=9
Scraping page: https://www.traknus.co.id/news/category/news?page=10
Scraped data from all pages saved to 'Traknus.csv'
                                          Title  \
0                    Genba CHCD 2024 at TRAKNUS   
1   TRAKNUS Group National Management Meetin...   
2    TRAKNUS Group Achieves 3 Bronze Winners...   
3   TRAKNUS Shows Concern Through Blood Dona...   
4   TRAKNUS Padang Strengthens

# Menggabungkan semua data set yang sudah di scraping

In [ ]:
# Define the paths to the previously downloaded CSV files
etana_csv = '/content/Etana.csv'
soho_csv = '/content/Soho.csv'
traknus_csv = '/content/Traknus.csv'

# Read each CSV file into a DataFrame
etana_df = pd.read_csv(etana_csv)
soho_df = pd.read_csv(soho_csv)
traknus_df = pd.read_csv(traknus_csv)

# Concatenate all DataFrames
tenant_df = pd.concat([etana_df, soho_df, traknus_df], ignore_index=True)

# Convert the 'Date' column to datetime format + TAMBAHAN
tenant_df['Date'] = pd.to_datetime(tenant_df['Date'], errors='coerce', dayfirst=True)

# Save to a single CSV file
tenant_df.to_csv('tenant_news.csv', index=False)
print("Scraped data from all websites saved to 'tenant_news.csv'")


Scraped data from all websites saved to 'tenant_news.csv'


In [ ]:
tenant_df.head(20)

,Title,Content,Source,Date
0,ETANA LUNCURKAN TERAPI KANKER INOVATIF DI INDO...,"Jakarta, 10 Desember 2024 – PT Etana Biotechno...",Etana,2024-12-10
1,ETANA DAN CELLTRION TANDATANGANI KESEPAKATAN S...,"Jakarta, 14 November 2024 – PT Etana Biotechno...",Etana,2024-11-14
2,"KUNJUNGI ETANA, BPOM DUKUNG INDUSTRI BIOTEKNOL...","Jakarta, 27 September 2024– Badan Pengawas Ob...",Etana,2024-09-27
3,ETANA BERSAMA DENGAN RSPAD GATOT SUBROTO SELEN...,"Jakarta, 12 Agustus 2024 – PT Etana Biotechnol...",Etana,2024-08-12
4,ETANA DUKUNG RS PENDIDIKAN UNIVERSITAS SYIAH K...,"Banda Aceh, 17 Juli 2024 – PT Etana Biotechnol...",Etana,2024-07-17
5,ETANA DAN PRIMAKU JALIN KERJASAMA KOLABORASI S...,"Jakarta, 29 April 2024 – PT Etana Biotechnolog...",Etana,2024-04-29
6,ETANA JALIN KERJA SAMA STRATEGIS DENGAN BEIGEN...,"Jakarta, 29 Januari 2024–PT Etana Biotechnolog...",Etana,2024-01-29
7,FASILITAS PRODUKSI VAKSIN mRNA ETANA TERIMA KU...,"Jakarta, 25 Oktober 2023 – PT Etana Biotechnol...",Etana,2023-10-25
8,ETANA PERKUAT KERJA SAMA BIOFARMASI DENGAN CAN...,"Jakarta, 16 Oktober 2023 - PT Etana Biotechnol...",Etana,2023-10-16
9,"ETANA PERKUAT KERJA SAMA DENGAN BRIN DAN UNSW,...","Jakarta, 22 September 2023 – PT Etana Biotechn...",Etana,2023-09-22


SORTED BASED ON LATEST NEWS

In [ ]:
# Define the paths to the previously downloaded CSV files
etana_csv = '/content/Etana.csv'
soho_csv = '/content/Soho.csv'
traknus_csv = '/content/Traknus.csv'

# Read each CSV file into a DataFrame
etana_df = pd.read_csv(etana_csv)
soho_df = pd.read_csv(soho_csv)
traknus_df = pd.read_csv(traknus_csv)

# Concatenate all DataFrames
tenant_df1 = pd.concat([etana_df, soho_df, traknus_df], ignore_index=True)

# Convert the 'Date' column to datetime format
tenant_df1['Date'] = pd.to_datetime(tenant_df1['Date'], errors='coerce', dayfirst=True)

# Sort the DataFrame by date in descending order
tenant_df1 = tenant_df1.sort_values(by='Date', ascending=False)

# Reset the index
tenant_df1.reset_index(drop=True, inplace=True)

# Save to a single CSV file
tenant_df1.to_csv('tenant_news_sorted.csv', index=False)
print("Scraped data from all websites saved to 'tenant_news_sorted.csv'")

Scraped data from all websites saved to 'tenant_news_sorted.csv'


In [ ]:
tenant_df1.head(20)

,Title,Content,Source,Date
0,ETANA LAUNCHES INNOVATIVE CANCER THERAPIES IN ...,"Jakarta, December 10, 2024 – PT Etana Biotechn...",Etana,2024-12-10
1,Genba CHCD 2024 at TRAKNUS,Traktor Nusantara (TRAKNUS) welcomed the arriv...,Traktor Nusantara,2024-12-10
2,TRAKNUS Group National Management Meetin...,With the theme “The Best Industrial & Agricult...,Traktor Nusantara,2024-12-09
3,TRAKNUS Group Achieves 3 Bronze Winners...,Traktor Nusantara (TRAKNUS) Group again record...,Traktor Nusantara,2024-12-06
4,TRAKNUS Shows Concern Through Blood Dona...,Traktor Nusantara (TRAKNUS) Group reaffirmed i...,Traktor Nusantara,2024-12-02
5,TRAKNUS Padang Strengthens Emergency Res...,Traktor Nusantara (TRAKNUS) in instilling a cu...,Traktor Nusantara,2024-11-28
6,ETANA AND CELLTRION SIGN STRATEGIC AGREEMENT F...,"Jakarta, 14 November 2024 – PT Etana Biotechno...",Etana,2024-11-14
7,TRAKNUS Mengajar Support Vocational Educ...,Traktor Nusantara (TRAKNUS) has 3 pillars that...,Traktor Nusantara,2024-10-22
8,TRAKNUS Builds Safety Awareness,As a form of firm commitment in maintaining he...,Traktor Nusantara,2024-10-21
9,Traktor Nusantara Gives MSME Training in...,Traktor Nusantara (TRAKNUS) in collaboration w...,Traktor Nusantara,2024-10-15
